In [ ]:
%load_ext autoreload
%autoreload 2
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from torch_scatter import scatter_add
import uproot
import awkward as ak

import sys
sys.path.append("..")
from dnn.ak_sample_loader import *
from dnn.torch_dataset import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
input = AkSampleLoader.loadFromPickle("/grid_mnt/data_cms_upgrade/cuisset/ticlRegression/PionSamples_v2/fullData.pkl")
dl = makeDataLoader(makeTrainingSample(input, "feat-v2", device="cpu"),  weighted=True, batch_size=1000, num_workers=5)

In [ ]:
list(dl)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb786fc4f40>
Traceback (most recent call last):
  File "/grid_mnt/data_cms_upgrade/cuisset/conda/envs/ticlRegression-gpu/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/grid_mnt/data_cms_upgrade/cuisset/conda/envs/ticlRegression-gpu/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/grid_mnt/data_cms_upgrade/cuisset/conda/envs/ticlRegression-gpu/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb786fc4f40>
Traceback (most recent call last):
  File "/grid_mnt/data_cms_upgrade/cuisset/conda/env

[{'features': tensor([[  8.9822,   2.4111, 350.2563,  ...,   0.0000,   0.0000,   0.0000],
          [  8.6334,   2.4369, 354.6375,  ...,   0.0000,   0.0000,   0.0000],
          [  2.2739,   2.4578, 365.3660,  ...,   0.0000,   0.0000,   0.0000],
          ...,
          [  3.4598,   2.3216, 386.5085,  ...,   0.0000,   0.0000,   0.0000],
          [ 37.7728,   2.9020, 333.1730,  ...,   0.0000,   0.0000,   0.0000],
          [  3.2605,   1.5894, 357.9842,  ...,   0.0000,   0.0000,   0.0000]]),
  'tracksterInEvent_idx': tensor([  0,   0,   0,  ..., 997, 998, 999]),
  'cp_energy': tensor([ 69.3392, 229.6146, 392.8523,  27.9886, 342.6495,  97.9305,  81.5084,
          294.4376, 125.4852, 120.6995, 358.5847, 137.3360, 123.8099,  38.4239,
           41.8770, 126.5665,  81.1847,  66.6370,  23.2929, 350.5187, 244.4729,
          228.0216, 216.0773,  50.7094,  73.1584,  29.9048, 299.6647,  99.9376,
           26.8333, 101.4993, 574.9900, 186.7743,  10.2252, 163.9359, 506.4955,
          350.6533

In [2]:
input = AkSampleLoader("/grid_mnt/data_cms_upgrade/cuisset/ticlRegression/PionSamples_v2/histo/histo_9376310_0.root")

In [3]:

ds = RegressionDataset(input, "feat-v2", 0, 500)

In [8]:
dl = makeDataLoader(makeValidationSample(input, "feat-v2", device="cpu"), batch_size=30, num_workers=5)

In [9]:
list(dl)

[{'features': tensor([[  3.7952,   2.3598, 409.7283,  ...,   0.0000,   2.8371,   0.0000],
          [  2.9154,   2.3868, 406.3126,  ...,   0.0000,   0.9756,   0.0000],
          [146.6855,   2.5863, 349.0024,  ...,   0.0000,   0.0000,   0.0000],
          ...,
          [ 12.2857,   1.6524, 395.9308,  ...,   0.0000,  12.2857,   0.0000],
          [ 72.2761,   1.9294, 352.4903,  ...,   0.0000,   3.6004,   0.0000],
          [  3.2235,   1.9493, 416.0695,  ...,   0.0000,   3.2235,   0.0000]]),
  'tracksterInEvent_idx': tensor([ 0,  0,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
           4,  5,  5,  5,  5,  5,  6,  6,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,
           8,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11,
          12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 14, 15, 15, 15, 15, 15, 16,
          16, 16, 16, 16, 16, 16, 16, 17, 17, 18, 18, 19, 20, 20, 20, 20, 20, 20,
          20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22,

In [36]:
dl = makeDataLoader(ds, batch_size=10, weighted=True)

In [37]:
batches = list(dl)
batches

[{'features': tensor([[3.7952e+00, 2.3598e+00, 4.0973e+02, 5.6846e+01, 1.6694e+00, 2.2043e-02,
           7.3238e+00, 1.7728e+00, 1.3252e+00, 5.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 9.5805e-01, 0.0000e+00, 2.8371e+00, 0.0000e+00],
          [2.9154e+00, 2.3868e+00, 4.0631e+02, 3.2293e+01, 1.2932e+00, 1.8214e-02,
           5.6809e+00, 1.1431e+00, 1.5931e-01, 4.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 1.9398e+00, 0.0000e+00, 9.7562e-01, 0.0000e+00],
          [6.5073e+00, 2.1655e+00, 3.3313e+02, 3.9679e+01, 4.8126e-01, 2.4120e-01,
           6.0599e+00, 7.0714e-01, 1.7831e+00, 2.3000e+01, 6.5073e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [7.3746e+01, 2.3424e+00, 3.9234e+02, 1.0280e+02, 1.8856e+00, 6.9775e-01,
           8.9226e+00, 3.3382e+00, 3.8239e+00, 1.7000e+01, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 4.6038e+01, 2.0341e+01, 7.3668e+00, 0.0000e+00],
          [7.1663e+01, 2.3568e+00, 3

In [40]:
batch_i = 8
scatter_add(batches[batch_i]["features"][:, 0], batches[batch_i]["tracksterInEvent_idx"])

tensor([ 35.5057,  28.5874, 136.4660, 262.6303,  41.2875,  13.3857,  29.4358,
         12.2857, 139.6178, 376.2262])

In [41]:
batches[batch_i]["cp_energy"]

tensor([ 77.1409,  51.2673, 261.0424, 449.4850,  90.1634,  27.3100,  66.5357,
         81.7647, 475.9894, 473.4459])

In [32]:
scatter_add(batches[batch_i]["features"][:, 0], batches[batch_i]["tracksterInEvent_idx"])/batches[batch_i]["cp_energy"]

tensor([0.2244, 0.5176, 0.6950, 0.4424])

In [31]:
input.makeDataAk()[0:2]

<Array [[[3.8, 2.36, ..., 2.84, 0], ...], ...] type='2 * var * 10 * float32'>

In [11]:
np.cumsum(ak.num(input.tracksters_splitEndcaps.raw_energy))

<Array [2, 8, 12, 18, 19, 24, ..., 419, 421, 425, 431, 435] type='94 * int64'>

In [16]:
input.tracksters_splitEndcaps.raw_energy

<Array [[3.8, 2.92], ..., [227, 11, 5.73, 2.07]] type='94 * var * float32'>

In [18]:
ak.flatten(input.tracksters_splitEndcaps.raw_energy, axis=-1)[431:435]

<Array [227, 11, 5.73, 2.07] type='4 * float32'>

In [15]:
input.makeDataAk("feat-v2")

<Array [[[3.8, 2.36, ..., 2.84, 0], ...], ...] type='94 * var * 17 * float64'>

In [16]:
input.caloparticles_splitEndcaps

<Array [{time: 13.2, ...}, {...}, ..., {...}] type='20 * caloparticle[time:...'>

In [8]:
# enum :   enum CellType {
#     CE_E_120 = 0,
#     CE_E_200 = 1,
#     CE_E_300 = 2,
#     CE_H_120 = 3,
#     CE_H_200 = 4,
#     CE_H_300 = 5,
#     CE_H_SCINT = 6,
#     EnumSize = 7
#   };


In [18]:
input.energyPerCellType

<Array [[[0, 0.153, ..., 0.141, 0], ...], ...] type='20 * var * 7 * float32'>

## Version without clipping

In [19]:
true_energy = torch.tensor(input.caloparticles_splitEndcaps.regressed_energy)

In [51]:
tsInEvt_idx = torch.tensor(ak.flatten(ak.broadcast_arrays(ak.local_index(input.tracksters_splitEndcaps.raw_energy, axis=0), input.tracksters_splitEndcaps.raw_energy)[0]))
tsInEvt_idx.shape

torch.Size([44])

In [42]:
def akToTensor_noClip(ar:ak.Array):
    return torch.tensor(ak.flatten(ar))

In [54]:
data = torch.stack([akToTensor_noClip(tracksters_splitEndcaps.raw_energy), akToTensor_noClip(input.tracksters_splitEndcaps.barycenter_eta), akToTensor_noClip(input.tracksters_splitEndcaps.barycenter_z)] + [akToTensor_noClip(input.energyPerCellType[:, :, cellType_i]) for cellType_i in range(7)], dim=-1)

In [57]:
data.shape

torch.Size([44, 10])

In [60]:
class NetworkNoClip(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(data.shape[-1], 1, dtype=data.dtype),
        )

    def forward(self, x):
        return torch.squeeze(self.model(x))
net = NetworkNoClip()

In [61]:
net(data)

tensor([ 29.8255,  32.7520,  28.7608,  27.9655,  36.0864,  28.8798,  32.2802,
         25.1754,  33.4956,  26.7288,  28.3129,  25.7363,  27.1157,  27.5983,
         29.3331,  28.9366,  25.8991,  35.6060,  28.1522,  28.6858,  31.3700,
         28.5225,  32.2580, -32.6915, -33.0454, -32.0779, -32.0959, -35.1889,
        -30.3262, -29.3601, -29.6520, -47.7183, -41.3281, -29.7770, -31.6684,
        -29.0842, -32.9314, -31.8502, -34.3254, -29.0414, -29.1650, -36.3884,
        -34.7929, -34.5013], grad_fn=<SqueezeBackward0>)

In [65]:
scatter_add(net(data), tsInEvt_idx)

tensor([  62.5775,   28.7608,   64.0518,   61.1601,   58.6711,   55.0416,
          25.7363,   27.1157,   85.8680,  210.4936,  -32.6915,  -97.2192,
         -35.1889,  -89.3383,  -89.0464,  -90.5297,  -64.7816,  -34.3254,
         -58.2064, -105.6825], grad_fn=<ScatterAddBackward0>)

In [66]:
loss_fct = nn.MSELoss()
loss = loss_fct(scatter_add(net(data), tsInEvt_idx), true_energy)
loss

tensor(7357.2197, grad_fn=<MseLossBackward0>)

In [69]:
scatter_add(data[:, 0], tsInEvt_idx)/true_energy

tensor([0.3364, 0.1629, 0.3169, 0.4494, 0.3576, 0.3785, 0.6750, 0.4717, 0.4358,
        0.4709, 0.8010, 0.6355, 0.8678, 0.3077, 0.6035, 0.3467, 0.5526, 0.7568,
        0.0697, 0.4638])